# 🚀 Vietnamese-English Translation Model Training

**Transformer-based Neural Machine Translation: VI → EN**

This notebook trains a **REVERSED** translation model:
- **Source**: Vietnamese 🇻🇳
- **Target**: English 🇬🇧

Using:
- **PhoMT Dataset** (~3M sentence pairs)
- **Modern Transformer** with RMSNorm, SwiGLU, RoPE
- **AdamW + Cosine Annealing**

---

## 1. ⚙️ Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# Install dependencies
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml

In [ ]:
# Verify PyTorch and CUDA
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')

In [ ]:
# Create .env file
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
    f.write(f'HF_TOKEN={HF_TOKEN}\n')

print('✓ .env file created')

## 2. 📥 Download & Preprocess Data

In [ ]:
# Download PhoMT dataset
!python scripts/download_phomt.py

In [ ]:
# ⚠️ IMPORTANT: Use VI-EN config for preprocessing!
# This will train Vietnamese tokenizer as SOURCE and English as TARGET

!python scripts/preprocess_data.py --config config/config_vi_en.yaml

## 3. 🔧 Configuration Check

In [ ]:
import sys
sys.path.insert(0, '.')

from src.config import load_config
from src.models import Transformer

# Load VI-EN config
config = load_config('config/config_vi_en.yaml')

print("="*50)
print("📋 Configuration Summary (VI → EN)")
print("="*50)
print(f"\n🔹 Direction: Vietnamese → English")
print(f"\n🔹 Model:")
print(f"   d_model: {config.d_model}")
print(f"   layers: {config.num_encoder_layers} enc + {config.num_decoder_layers} dec")
print(f"   d_ff: {config.d_ff}")
print(f"   dropout: {config.dropout}")
print(f"\n🔹 Training:")
print(f"   epochs: {config.epochs}")
print(f"   batch_size: {config.batch_size}")
print(f"   learning_rate: {config.learning_rate}")
print(f"   warmup_steps: {config.warmup_steps}")
print(f"\n🔹 Data:")
print(f"   src_vocab (VI): {config.src_vocab_size}")
print(f"   tgt_vocab (EN): {config.tgt_vocab_size}")
print(f"   checkpoint_dir: {config.paths.checkpoint_dir}")

In [ ]:
# Count model parameters
model = Transformer(
    src_vocab_size=config.src_vocab_size,
    tgt_vocab_size=config.tgt_vocab_size,
    d_model=config.d_model,
    num_heads=config.num_heads,
    num_encoder_layers=config.num_encoder_layers,
    num_decoder_layers=config.num_decoder_layers,
    d_ff=config.d_ff
)

total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Parameters: {total_params:,} ({total_params/1e6:.1f}M)")
print(f"📦 Model Size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

## 4. 🏋️ Training

In [ ]:
# Switch to processed data for faster training
import yaml

with open('config/config_vi_en.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['source'] = 'processed'

with open('config/config_vi_en.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config updated to use processed data')

### ⚡ Resume Training from Checkpoint (Optional)

In [ ]:
#@title 📂 Nhập đường dẫn checkpoint để resume training

import os

# ========================================
# 👇 NHẬP ĐƯỜNG DẪN CHECKPOINT TẠI ĐÂY 👇
# ========================================
RESUME_CHECKPOINT_PATH = ""  # Ví dụ: "/kaggle/input/my-vi-en-model/best_model.pt"

if RESUME_CHECKPOINT_PATH and os.path.exists(RESUME_CHECKPOINT_PATH):
    print(f"✅ Checkpoint found: {RESUME_CHECKPOINT_PATH}")
    RESUME_FROM = RESUME_CHECKPOINT_PATH
else:
    if RESUME_CHECKPOINT_PATH:
        print(f"⚠️ Checkpoint NOT found: {RESUME_CHECKPOINT_PATH}")
    print("📌 Will train from scratch")
    RESUME_FROM = None

In [ ]:
# Start training!
from src.train import Trainer

# ⚠️ IMPORTANT: Use VI-EN config!
trainer = Trainer(config_path='config/config_vi_en.yaml')
trainer.setup()

print("\n" + "="*50)
print("🇻🇳 → 🇬🇧 Vietnamese to English Translation")
print("="*50)

if RESUME_FROM:
    print(f"\n🔄 Resuming training from: {RESUME_FROM}")
    trainer.train(resume_from=RESUME_FROM)
else:
    print("\n🚀 Starting training from scratch")
    trainer.train()

## 5. 📊 Evaluation

In [ ]:
# Evaluate VI->EN model
!python -m src.evaluate \
    --checkpoint checkpoints_vi_en/best_model.pt \
    --vocab-src checkpoints_vi_en/tokenizers/tokenizer_vi.model \
    --vocab-tgt checkpoints_vi_en/tokenizers/tokenizer_en.model \
    --config config/config_vi_en.yaml \
    --test

## 6. 🔮 Interactive Translation

In [ ]:
from src.evaluate import load_translator

# Load VI->EN model
translator = load_translator(
    checkpoint_path='checkpoints_vi_en/best_model.pt',
    vocab_src_path='checkpoints_vi_en/tokenizers/tokenizer_vi.model',
    vocab_tgt_path='checkpoints_vi_en/tokenizers/tokenizer_en.model',
    config_path='config/config_vi_en.yaml'
)

In [ ]:
# Test Vietnamese -> English translation!
test_sentences = [
    "Xin chào, bạn khỏe không?",
    "Hôm nay thời tiết rất đẹp.",
    "Tôi thích học ngôn ngữ mới.",
    "Dịch máy đang phát triển rất nhanh.",
    "Bạn có thể giúp tôi được không?"
]

print("="*60)
print("🌐 Translation Examples (VI → EN)")
print("="*60)

for sentence in test_sentences:
    translation = translator.translate(sentence, beam_size=4)
    print(f"\n🔹 VI: {sentence}")
    print(f"🔹 EN: {translation}")

## 7. 💾 Save Model to Kaggle Output

In [ ]:
import shutil
import os

# Copy VI-EN model files to Kaggle output
output_dir = '/kaggle/working/model_output_vi_en'
os.makedirs(output_dir, exist_ok=True)

# Copy checkpoint
shutil.copy('checkpoints_vi_en/best_model.pt', output_dir)

# Copy tokenizers
shutil.copytree('checkpoints_vi_en/tokenizers', f'{output_dir}/tokenizers', dirs_exist_ok=True)

# Copy config
shutil.copy('config/config_vi_en.yaml', output_dir)

# Copy logs
if os.path.exists('logs_vi_en'):
    shutil.copytree('logs_vi_en', f'{output_dir}/logs', dirs_exist_ok=True)

print(f'\n✓ VI-EN Model saved to {output_dir}')
print('\nFiles:')
for f in os.listdir(output_dir):
    print(f'  - {f}')

---

## 📝 Notes

**Model Info (VI → EN):**
- Architecture: Modern Transformer (SwiGLU, RMSNorm, RoPE)
- Direction: Vietnamese → English
- Encoder layers: 6 (more for complex Vietnamese)
- Decoder layers: 5 (less for simpler English)

**Key Differences from EN→VI:**
- Source language is Vietnamese (more complex tokenization)
- Target language is English (simpler structure)
- Different tokenizers trained for each direction
- Separate checkpoint folder (`checkpoints_vi_en/`)

**Tips:**
1. Vietnamese sentences may need more encoder capacity
2. English output is generally simpler to generate
3. BLEU scores may differ from EN→VI direction